<a href="https://colab.research.google.com/github/Andrey-Epifanov-k721/NLP_DLS/blob/main/%5Bhomework_1%5Dsimple_embeddings_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [9]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2023-06-17 14:57:30--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.124.72
Connecting to zenodo.org (zenodo.org)|188.185.124.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  24.1MB/s    in 60s     

2023-06-17 14:58:31 (23.3 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [10]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [11]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [12]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слов `cat` топ-5 близких слов к слову `dog`? Какое место?

In [13]:
# method most_simmilar
word = 'dog'
if word in wv_embeddings:
    similar_words = wv_embeddings.most_similar(word, topn=5)
    similar_words = [w[0] for w in similar_words]
    if 'cat' in similar_words:
        position = similar_words.index('cat') + 1
        print(f"'cat' входит в топ-5 близких слов к '{word}' на позиции {position}")
    else:
        print("'cat' не входит в топ-5 близких слов к", word)
else:
    print(word, "не найдено в модели Word2Vec")


'cat' не входит в топ-5 близких слов к dog


In [14]:
if word in wv_embeddings:
    similar_words = wv_embeddings.most_similar(word, topn=50)
    similar_words = [w[0] for w in similar_words]
    if 'cat' in similar_words:
        position = similar_words.index('cat') + 1
        print(f"'cat' входит в топ-5 близких слов к '{word}' на позиции {position}")

'cat' входит в топ-5 близких слов к 'dog' на позиции 26


### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [15]:
import numpy as np
import re
# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)    # используется регулярное выражение '\w+', которое ищет последовательности символов, состоящие из букв, цифр и знака подчеркивания.
tokenizer = MyTokenizer()

In [ ]:
# def question_to_vec(question, embeddings, tokenizer, dim=200):
#     """
#         question: строка
#         embeddings: наше векторное представление
#         dim: размер любого вектора в нашем представлении

#         return: векторное представление для вопроса
#     """

#     '''your code'''

Функция question_to_vec преобразует вопрос в его векторное представление путем вычисления среднего вектора из векторных представлений всех токенов, присутствующих в словаре embeddings. Если вопрос не содержит известных токенов, то возвращается нулевой вектор.

In [16]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    tokens = tokenizer.tokenize(question.lower()) # токенизация с предварительным понижением регистра
    vectors = []                                  # пустой список для векторного представления для каждого токена из вопроса

    # Если токен присутствует в embeddings (словаре векторных
    # представлений), то его векторное представление добавляется в список vectors.
    for token in tokens:
        if token in embeddings:
            vectors.append(embeddings[token])
    # роверяется длина списка vectors. Если он пустой, то возвращается
    # нулевой вектор размерности dim,
    if len(vectors) == 0:
        return np.zeros(dim)
    # Если список vectors содержит векторные представления, то вычисляется
    # и возвращается их среднее значение.
    return np.mean(vectors, axis=0)

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [17]:
sentence = "I love neural networks"
vector = question_to_vec(sentence, wv_embeddings, tokenizer)
third_component = round(vector[2], 2)
print(third_component)

-1.29


### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

Косинусное расстояние между предложениями - это метрика, которая измеряет сходство или различие между двумя предложениями, основываясь на угле между их векторными представлениями. Оно вычисляется путем измерения косинуса угла между двумя векторами предложений.

Косинусное расстояние является нормированным значением и находится в диапазоне от 0 до 1, где 0 означает полное различие (предложения не имеют общих элементов), а 1 означает полное сходство (предложения идентичны).

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [18]:
def hits_count(dup_ranks, k):
    """
    dup_ranks: список рангов дубликатов
    k: значение k для вычисления Hits@k
    """
    hits_value = sum(1 for rank in dup_ranks if rank < k)
    return hits_value

In [19]:
def dcg_score(dup_ranks, k):
    """
    dup_ranks: list of duplicate ranks
    k: value of k for DCG@k calculation
    """
    dcg_value = sum(1 / np.log2(rank + 2) for rank in dup_ranks if rank < k)
    return dcg_value

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [20]:
import pandas as pd

In [21]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наги кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [i for i, candidate in enumerate(candidates_ranking[0]) if candidate in copy_answers]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0, 1, 1, 1]
Ваш ответ DCG: [0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [22]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!unzip /content/drive/MyDrive/stackoverflow_similar_questions.zip

Archive:  /content/drive/MyDrive/stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [25]:
def read_corpus(filename):
    data = []
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(line.strip())
    return data

Нам понадобиться только файл validation.

In [26]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [27]:
len(validation_data)

3760

Размер нескольких первых строк

In [28]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 54516
2 52817
3 54463
4 53723
5 52296


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [30]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    question_vector = question_to_vec(question, embeddings, tokenizer, dim)
    candidate_vectors = [question_to_vec(candidate, embeddings, tokenizer, dim) for candidate in candidates]
    similarities = cosine_similarity([question_vector], candidate_vectors)[0]
    ranked_candidates = sorted(enumerate(candidates), key=lambda x: similarities[x[0]], reverse=True)
    return ranked_candidates

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [31]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [32]:
for question, q_candidates in zip(questions, candidates):
        ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
        print(ranks)
        print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results'), (1, 'WPF- How to update the changes in list item of a list')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [ ]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(*, 'Getting all list items of an unordered list in PHP'), # скрыт
            (*, 'select2 not displaying search results'), #скрыт
            (*, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

021

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [35]:
from tqdm.notebook import tqdm

In [36]:
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [37]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 234.706 | Hits@   5: 372.000
DCG@  10: 234.706 | Hits@  10: 372.000
DCG@ 100: 234.706 | Hits@ 100: 372.000
DCG@ 500: 235.097 | Hits@ 500: 375.000
DCG@1000: 235.406 | Hits@1000: 378.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [38]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [51]:
from gensim.models import Word2Vec

# Concatenate question and candidate pairs
questions = ['converting string to list', 'Sending array via Ajax fails']
candidates = [
    [
        'Convert Google results object (pure js) to Python object',
        'C# create cookie from string and send it',
        'How to use jQuery AJAX for an outside domain?'
    ],
    [
        'Getting all list items of an unordered list in PHP',
        'WPF- How to update the changes in list item of a list',
        'select2 not displaying search results'
    ]
]
pairs = [q + candidate for q, candidates in zip(questions, candidates) for candidate in candidates]

words = [pair.split() for pair in pairs]

# Train Word2Vec model
embeddings_trained = Word2Vec(words,  # data for training the model
                              vector_size=200,  # embedding vector size
                              min_count=5,  # consider words that occurred at least 5 times
                              window=5).wv  # window size


# Rest of the code remains the same
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))


  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 630.930 | Hits@   5: 1000.000
DCG@  10: 630.930 | Hits@  10: 1000.000
DCG@ 100: 630.930 | Hits@ 100: 1000.000
DCG@ 500: 630.930 | Hits@ 500: 1000.000
DCG@1000: 630.930 | Hits@1000: 1000.000


In [54]:
from gensim.models import Word2Vec

train_data = read_corpus('./data/train.tsv')

# Concatenate question and candidate pairs
pairs = [question + candidate for question, candidates in zip(questions, candidates) for candidate in candidates]

words = [pair.split() for pair in pairs]

# Combine train_data with pairs
train_words = words + train_data

# Train Word2Vec model
embeddings_trained = Word2Vec(train_words,  # data for training the model
                              vector_size=200,  # embedding vector size
                              min_count=5,  # consider words that occurred at least 5 times
                              window=5).wv  # window size

# Rest of the code remains the same
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))



  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.000 | Hits@   1: 0.000
DCG@   5: 4.417 | Hits@   5: 7.000
DCG@  10: 4.417 | Hits@  10: 7.000
DCG@ 100: 4.417 | Hits@ 100: 7.000
DCG@ 500: 4.782 | Hits@ 500: 10.000
DCG@1000: 4.884 | Hits@1000: 11.000


In [55]:
# Rest of the code remains the same
wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

  0%|          | 0/3760 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:
